In [ ]:
!pip install transformers -q
!pip install datasets -q
!pip install sentencepiece -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from tqdm import tqdm

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bogdancazan/pegasus-text-simplification_1e4_adafactor_wikilarge_20epici")

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("bogdancazan/pegsaus-base-newsela-biendata-with-domain-adaptation")

In [ ]:
from datasets import load_dataset
dataset = load_dataset('bogdancazan/news-not-not-ela-text-simplification')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/bogdancazan___csv/bogdancazan--news-not-not-ela-text-simplification-cdb0ef56704f4c59/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

In [ ]:
normal_list, simple_list, generated_list = [], [], []
for entry in tqdm(dataset['test']):
  normal = entry['Normal']
  simple = entry['Simple']
  normal_list.append(normal)
  simple_list.append(simple)

  generated_ids = tokenizer(normal, max_length=80, padding=True, truncation=True, return_tensors="pt").input_ids

  generated_outputs = model.generate(generated_ids,
              max_length=80,
              num_beams=4,
              repetition_penalty=2.5,
              length_penalty=1.0,
              early_stopping=True
        )
  generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
  generated_list.append(generated_text)

d = {'Normal' : normal_list, 'Simple': simple_list, 'Generated': generated_list}

 12%|█▏        | 175/1414 [33:00<3:32:21, 10.28s/it]

In [ ]:
predictions_df = pd.DataFrame(d)
predictions_df.to_csv('predictions.csv')

In [ ]:
predictions_df

In [ ]:
nr = 0
for i in range(len(predictions_df)):
  if predictions_df['Normal'].iloc[i] == predictions_df['Generated'].iloc[i]:
    nr += 1
print(nr)